In [1]:
# pulled from
# https://cryptomarketpool.com/use-the-graph-to-query-ethereum-data-in-python/

import requests
from pprint import pprint


def run_query(q):

    # endpoint where you are making the request
    request = requests.post('https://api.thegraph.com/subgraphs/name/blocklytics/bancor',
                            '',
                            json={'query': q})
    if request.status_code == 200:
        return request.json()
    else:
        raise Exception('Query failed. return code is {}.      {}'.format(request.status_code, query))
        

def get_token_query(tkn):
    token_query = """
    {
      tokens(where: {symbol: """ + '"' + tkn + '"' + """}) {
        id
        decimals
      }
    }

    """
    return token_query

    
token_dict = {}
def get_token_info(tkn):
    if tkn not in token_dict:
        q = get_token_query(tkn)
        token_dict[tkn] = run_query(q)['data']['tokens'][0]
    return token_dict[tkn]
eth_id = get_token_info('ETH')['id']


# The Graph query - Query aave for a list of the last 10 flash loans by time stamp
# last day: 1642900000




query = """

{
  swaps(orderBy: timestamp, orderDirection: desc, where: {timestamp_gte: 1640000000}) {
    id
    timestamp
    fromToken {
      symbol
    }
    toToken {
      symbol
    }
    amountPurchased
    amountReturned
  }
}
"""

query = """

{
  swaps(orderBy: timestamp, orderDirection: desc, where: {fromToken: """ + '"' + eth_id + '"' + """}, or: {toToken: """ + '"' + eth_id + '"' + """}
  ) {
    id
    timestamp
    fromToken {
      symbol
    }
    toToken {
      symbol
    }
    transaction {
      blockNumber
    }
    amountPurchased
    amountReturned
  }
}
"""


swaps_raw = run_query(query)['data']['swaps']
swaps = [None] * len(swaps_raw)
print(len(swaps_raw))
for i in range(len(swaps_raw)):
    from_decimals = get_token_info(swaps_raw[i]['fromToken']['symbol'])['decimals']
    to_decimals = get_token_info(swaps_raw[i]['toToken']['symbol'])['decimals']
    swaps[i] = {
        'fromToken': swaps_raw[i]['fromToken']['symbol'],
        'toToken': swaps_raw[i]['toToken']['symbol'],
        'amountPurchased': int(swaps_raw[i]['amountPurchased'])/(10 ** from_decimals),
        'amountReturned': int(swaps_raw[i]['amountReturned'])/(10 ** to_decimals),
        'blockNumber': swaps_raw[i]['transaction']['blockNumber'],
        'timestamp': swaps_raw[i]['timestamp']
    }
    if i % 10 == 0:
        print(i)

# print the results
#print('Print Result - {}'.format(result))
#print('#############')
# pretty print the results to make it easier to read
pprint(swaps)
pprint(token_dict)

100
0
10
20
30
40
50
60
70
80
90
[{'amountPurchased': 0.1,
  'amountReturned': 0.013421758521653435,
  'blockNumber': '10890025',
  'fromToken': 'ETH',
  'timestamp': '1600483522',
  'toToken': 'eXAU'},
 {'amountPurchased': 0.1,
  'amountReturned': 0.01383582950730637,
  'blockNumber': '10889494',
  'fromToken': 'ETH',
  'timestamp': '1600476604',
  'toToken': 'eXAU'},
 {'amountPurchased': 0.06952545355030906,
  'amountReturned': 19.204495886289394,
  'blockNumber': '10276751',
  'fromToken': 'ETH',
  'timestamp': '1592309955',
  'toToken': 'BNT'},
 {'amountPurchased': 30.0,
  'amountReturned': 8300.024285808751,
  'blockNumber': '10276732',
  'fromToken': 'ETH',
  'timestamp': '1592309752',
  'toToken': 'BNT'},
 {'amountPurchased': 7.5,
  'amountReturned': 2079.1829599551224,
  'blockNumber': '10276727',
  'fromToken': 'ETH',
  'timestamp': '1592309684',
  'toToken': 'BNT'},
 {'amountPurchased': 10.0,
  'amountReturned': 2774.14777643122,
  'blockNumber': '10276717',
  'fromToken': 'E

In [2]:
from datetime import datetime
ts = int('1642900000')

# if you encounter a "year is out of range" error the timestamp
# may be in milliseconds, try `ts /= 1000` in that case
print(datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S'))

2022-01-23 01:06:40
